In [14]:
from openai import OpenAI
import glob
import tiktoken
import json
import requests
api_key = open("api_key", "r").read()
client = OpenAI(api_key=api_key)

def request_gpt4(messages, response_format=None):
    # enc = tiktoken.encoding_for_model("gpt-4-1106-preview")
    enc = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")
    text = json.dumps(messages)
    # print(len(enc.encode(text)))
    kept_index = 0
    while len(enc.encode(text)) > 16385:
    # while len(enc.encode(text)) > 128000:
        print("truncating...")
        # find the first user input
        for index, message in enumerate(messages):
            if message['role'] == 'user' and len(message['content']) > 1000:
                messages[index] = {
                    "role": "user",
                    "content": message['content'][:-1000]
                }
                break
        text = json.dumps(messages)
    try:
        # print(len(enc.encode(text)))
        if response_format == "json":
            response = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=messages,
                response_format={ "type": "json_object" },
                temperature=0.5
            )
        else:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=messages,
                temperature=0.5
            )
    except Exception as e:
        print(e)
        print("retrying...")
        return request_gpt4(messages)
    return response.choices[0].message.content

def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(data, fp, indent=4)

# def get_embedding(text, model="text-embedding-ada-002"):
#     enc = tiktoken.encoding_for_model(model)
#     while len(enc.encode(text)) > 8191:
#         text = text[:-100]
#     url = 'https://api.openai.com/v1/embeddings'
#     headers = {
#         'Content-Type': 'application/json',
#         'Authorization': "Bearer {}".format(api_key)
#     }
#     data = {
#         "input": text,
#         "model": model
#     }
#     res = requests.post(url, headers=headers, json=data)
#     res = res.json()
#     return res['data'][0]['embedding']

def cosine_similarity(a, b):
    from numpy import dot
    from numpy.linalg import norm
    return dot(a, b)/(norm(a)*norm(b))


In [15]:
def extract_keywords(paragraph):
    messages = [
        {
            "role": "system",
            "content": """You are a keyword extraction system that extracts keywords from a monologue. 
            The monologue is about a person who is talking about their life.
            The keywords should be the most important words in the monologue.
            Use the exact words that the person uses in the monologue.
            Replay with a list of keywords in {{language}}.
            """.format(language="English")
       },
       {
           "role": "user",
           "content": paragraph
       }
    ]
    keywords = request_gpt4(messages)
    return keywords

In [16]:
# Note: run time very long, consider running on kwon
for interview_file in glob.glob("../data/result/1203/chunk_summaries/*.json"):
    interview_data = json.load(open(interview_file))
    for chunk in interview_data:
        interviewee_messages = "\n".join([message['content'] for message in chunk['conversation'] if message['speaker'] == 0])
        keywords = extract_keywords(interviewee_messages)
        print(interviewee_messages)
        print(keywords)
    break

3176
基本上就是那反彈的內容，我覺得其實也沒有什麼，我來講沒有什麼好就是不公開的，然後就是只要就是有要使用的話，就是訊息跟我講一下，基本上就OK這樣。
因為之前的工作裡面，其實我也是從就是研究生才在綠島的。
然後不小心就太喜歡這裡邊，然後就留下來了。
我當時其實是針居民做訪談。
，然後當時其實也是因為居民很友善的關係，主要應該也是因為。 我就自己一個人來做研究，所以其實這邊長輩，就一個女生，然後就是來做研究，然後還要把就是一些中文很難懂的東西，就是翻譯成閩南語，跟他們溝通有很多可能是就是不識字的，那他要聽你解釋，因為他們不會輕易簽任何的東西，即使你做不具名。因為在以前那個綠島的社會裡面，可能會有一些像不動產、土地上的一些就是被騙簽名之類。
他們會還有這樣的就是記憶在，所以其實有時候訪談，其實我來到這邊一開始就是有很照顧的長輩，但也有比較擔心害怕就是把我給趕出去，這樣也是會有。
我是住環境，就是跟環境相關。因為其實我原本是想要做一個離島工藝的，一個議題就是你應該要投入什麼樣工藝行為才是符合離島的，但我發現其實我當時還在做十幾年前，我在十幾年前就來了過程裡面，其實會覺得第一是文獻少，然後第二是其實那時候還沒有在推那個社會企業責任，但是已經有部分的公司來做，但是相關數據太少，你要去支持這件事情跟當時的綠島的風氣，其實是一個商業起飛的年代。其實大家過得賺錢，其實比較會去忽略掉這些部分，甚至是我在這邊在做的時候，有一些可能是家裡面比較大的算是企業。企業主要是反正要大一點的規模的飯店，他們會直接告訴我，他說：「你在談離島工藝，然後說那你知道我們綠島一天的產值是多少，然後一天可能就1000萬以上，你覺得我們離島需要嗎？ ，然後當時其實有震撼到，我就覺得偏鄉離島好像很需要什麼樣的經費注入。但其實說真的，就是每一個在學術上研究的絕不比在地的商人，他會一定會先看到什麼樣的商機，已經有奠定這樣的基礎，所以我那時候就回去好好就研究這件事情，然後再去跟這邊的長輩耆老就是也是聊天，然後我當時其實要做研究以前，我現在這邊工讀大概一個多月，然後就是我的工作都是跟環境相關的，後來才決定轉換題目。
{研究生, 綠島, 訪談, 中文, 閩南語, 社會企業, 離島工藝, 商業, 反彈, 長輩, 環境, 學術, 商機, 奠定基礎, 耆老, 工讀, 轉換題目}
1797
我原本是念醫學類醫學，但是因為我發

KeyboardInterrupt: 

In [52]:
from pprint import pprint

def generate_nodes(paragraph,definition_dict, var_type, chunk_id, conversation_id):
    definitions = [f"{index+1}. {var}: {definition_dict[var]}" for index, var in enumerate(definition_dict.keys())]
    var_list = [var for var in definition_dict.keys()]
    # print(definitions)
    # print(var_list)
    var_definitions = "\n".join(definitions)
    
    # user_content = """<paragraph> {paragraph} </paragraph> \n <{var_type}_definitions> {var_definitions} </{var_type}_definitions>""".format()
    # user_content2 = """Paragraph: {paragraph} \n {var_type} definitions: {var_definitions}""".format(paragraph=paragraph, var_type="Drivers", var_definitions=var_definitions)
    # str_template = "This is a {var_type} definition: {definition}".format(var_type="Drivers", definition="Drviers are ....")

    #驅動變數是基本的人為原因，引起環境中的某些影響，以滿足基本的人類需求。
    #壓力是對環境或生態系統的負面現象或活動，這些是由驅動變數引起的或自然發生的。
    #狀態指的是特定時間框架和區域內的物理、化學和生物現象的數量和質量。
    #影響指的是環境條件、生態系統功能或人類福祉方面的不良變化。 
    #回應指的是保護環境、應對環境問題或友善環境的任何行為、行動或努力。

    messages = [
        {
            "role":"system",
            "content":""" You are a Driver variables extraction system that extracts nodes from a monologue.
            You will be given a monologue, which is about a person who is talking about their life.
            There are five categories of variables: Drivers, Pressures, State, Impacts, and Responses.
            The user is interested in the {var_type} mentioned in the monoglogue. 
            This is a {var_type} definition: "Pressures are negative phenomena or activities to the environment or ecosystem that Drivers cause or are naturally occurring."
            You will be provided with a monologue in Chinese , and a list contains both the driver variables and the definitions of each driver variables in Chinese.
            
            First, you will need to identify the variables that belongs to {var_type} category and are mentioned in or might be related to the monologue.
            Then, for each variable, extract the sentence or sentences that mention it.
            Note that there might be no {var_type} variable mentioned in the monologue.
            And that variables must be in the list of variables.
            Please provide output in JSON format as follows:
            {{
                "variable_mentions": [
                {{
                    "variable": (one of the {var_type} in the provided list of definitions),
                    "mentions": {{"text": (string of the sentence)}},
                }},
                {{
                    "variable": (one of the {var_type} in the provided list of definitions),
                    "mentions": {{"text": (string of the sentence)}},
                }},
                ...
                {{
                    "variable": (one of the {var_type} in the provided list of definitions),
                    "mentions": {{"text": (string of the sentence)}},
                }},
                {{
                    "variable": (one of the {var_type} in the provided list of definitions),
                    "mentions": {{"text": (string of the sentence)}},
                }},
            ]
        }}
            
            """.format(var_type=var_type, var_definitions=var_definitions)
        },
        {
            "role":"user",
            "content":"Monologue: {paragraph} \n List of definitions:{var_definitions} \n List of variables:{var_list}".format(paragraph=paragraph,var_definitions=var_definitions, var_list=var_list)
        },
    ]
    pprint(messages)
    response = request_gpt4(messages, response_format="json")
    pprint(response)
    # json_string = nodes.strip('`').strip('json')
    # print(json_string)
    nodes_object = json.loads(response)['variable_mentions']

    for object in nodes_object:
        object["chunk_id"] = chunk_id
        object["conversation_id"] = conversation_id

    return nodes_object


In [29]:
from pprint import pprint

def generate_nodes(paragraph, var_type_def, definition_dict, var_type, chunk_id, conversation_id):
    definitions = [f"{index+1}: {var}. Examples: {definition_dict[var]}" for index, var in enumerate(definition_dict.keys())]
    var_list = [var for var in definition_dict.keys()]
    # print(definitions)
    # print(var_list)
    var_definitions = "\n".join(definitions)
    var_num = len(definition_dict.keys())
    # user_content = """<paragraph> {paragraph} </paragraph> \n <{var_type}_definitions> {var_definitions} </{var_type}_definitions>""".format()
    # user_content2 = """Paragraph: {paragraph} \n {var_type} definitions: {var_definitions}""".format(paragraph=paragraph, var_type="Drivers", var_definitions=var_definitions)
    # str_template = "This is a {var_type} definition: {definition}".format(var_type="Drivers", definition="Drviers are ....")

    #驅動變數是基本的人為原因，引起環境中的某些影響，以滿足基本的人類需求。
    #壓力是對環境或生態系統的負面現象或活動，這些是由驅動變數引起的或自然發生的。
    #狀態指的是特定時間框架和區域內的物理、化學和生物現象的數量和質量。
    #影響指的是環境條件、生態系統功能或人類福祉方面的不良變化。 
    #回應指的是保護環境、應對環境問題或友善環境的任何行為、行動或努力。

    messages = [
        {
            "role":"system",
            "content":""" You are a {var_type} variables extraction system that extracts nodes from a monologue.
            A {var_type} is defined as: {var_type_def}.
            You will be given a monologue, which is about a person who is talking about their life.
            The user is interested in the {var_type} mentioned in the monoglogue, which includes the following objects:
            {var_definitions}
            The user will give you a monologue in Chinese.
            You will need to identify the objects that are related to the monologue.
            All object names must match one of the {var_num} objects in this list: {var_list}.
            DO NOT MAKE UP OBJECTS NOT IN THE LIST.
            Note that some object might not be mentioned in the monologue.
            Please provide output in JSON format as follows:
            {{
                "objects": [] (objects from the object list)
            }}
            """.format(var_type=var_type, var_definitions=var_definitions, var_num=var_num, var_type_def=var_type_def, var_list=var_list)
        },
        {
            "role":"user",
            # "content":"Monologue: {paragraph} \n List of definitions:{var_definitions} \n List of variables:{var_list}".format(paragraph=paragraph,var_definitions=var_definitions, var_list=var_list)
            "content":"Monologue: {paragraph}.  ".format(paragraph=paragraph,var_definitions=var_definitions)
        },
    ]
    # pprint(messages)
    response = request_gpt4(messages, response_format="json")
    # pprint(response)
    # json_string = nodes.strip('`').strip('json')
    # print(json_string)
    nodes_object = json.loads(response)['objects']
    # print(nodes_object)
    # errors = list(filter(lambda object: object not in var_list, nodes_object))
    # if len(errors) > 3:
    #     print("Too many errors, retrying...")
    #     return generate_nodes(paragraph, var_type_def, definition_dict, var_type, chunk_id, conversation_id)
    # else:
    nodes_object = list(filter(lambda object: object in var_list, nodes_object))
    # print(nodes_object)
    result = []
    # res["conversation_id"] = conversation_id
    for object in nodes_object:
        result.append({
            "var": object,
            "chunk_id": chunk_id,
            "mentions": []
        })

    
    return result

In [38]:
from collections import defaultdict
variable_type = "Drivers"
variable_defs = json.load(open("../data/raw/definitions/{}_variables.json".format(variable_type), 'r', encoding='utf-8'))
var_type_def = "驅動變數是基本的人為原因，引起環境中的某些影響，以滿足基本的人類需求。"
var_list = [var for var in variable_defs.keys()]
result = {
    "variable_type": variable_type,
    "variable_mentions": defaultdict(list)
}
for var in var_list:
    result["variable_mentions"][var] = {
        "var_name": var,
        "mentions": []
    }
print(variable_defs.keys())
# with open("../data/raw/definitions/Pressures_variables.json", 'r', encoding='utf-8') as file:
#     var_def = json.load(file)
for interview_file in glob.glob("../data/result/1203/chunk_summaries/*.json"):
    interview_data = json.load(open(interview_file))
    for chunk in interview_data:
        chunk_id = chunk['id']
        conversation_id = 0
        interviewee_messages = "\n".join([message['content'] for message in chunk['conversation'] if message['speaker'] == 0])
        nodes_object = generate_nodes(interviewee_messages, var_type_def, variable_defs, variable_type, chunk_id, conversation_id)
        # print(nodes_object)
        for object in nodes_object:
            # print(object)
            result["variable_mentions"][object['var']]['mentions'].append(object)
        # for message in chunk['conversation']:
        #     if message['speaker'] == 0:
        #         # set the first conversation of interviewee to 1
        #         conversation_id = conversation_id+1
        #         interviewee_messages = message['content']
        #         nodes_object = generate_nodes(interviewee_messages, variable_defs, variable_type, chunk_id, conversation_id)
        #         all_objects.extend(nodes_object)
        
        # pprint(all_objects)
    # save_json(all_objects, "../data/result/Drivers_nodes.json")
    pprint(result)        
    break
    

    

dict_keys(['旅遊業', '漁業', '人口', '農業', '城市化', '交通運輸', '經濟', '沿海發展', '能源', '工業化', '住房', '健康', '基礎設施發展'])
[]
['旅遊業', '人口', '經濟', '健康', '基礎設施發展']
['漁業']
['漁業', '旅遊業', '經濟', '健康']
[]
['漁業']
[]
['經濟']
['經濟', '漁業']
['經濟', '城市化', '健康']
['旅遊業', '農業', '健康']
{'variable_mentions': defaultdict(<class 'list'>,
                                  {'交通運輸': {'mentions': [], 'var_name': '交通運輸'},
                                   '人口': {'mentions': [{'chunk_id': 'N10_background_1',
                                                        'mentions': [],
                                                        'var': '人口'}],
                                          'var_name': '人口'},
                                   '住房': {'mentions': [], 'var_name': '住房'},
                                   '健康': {'mentions': [{'chunk_id': 'N10_background_1',
                                                        'mentions': [],
                                                        'var': '健康'},
                        

In [33]:
save_json(result, "../data/result/Drivers_nodes.json")

In [ ]:
# Note: run time very long, consider running on kwon
for interview_file in glob.glob("../data/result/1203/chunk_summaries/*.json"):
    interview_data = json.load(open(interview_file))
    print(interview_file)
    for chunk in interview_data:
        if 'keywords' in chunk:
            continue
        interviewee_messages = "\n".join([message['content'] for message in chunk['conversation'] if message['speaker'] == 0])
        keywords = extract_keywords(interviewee_messages)
        print(interviewee_messages)
        print(keywords)
        chunk['keywords'] = keywords
    save_json(interview_data, interview_file)

../data/result/1203/chunk_summaries\N10_background.json
../data/result/1203/chunk_summaries\N10_topics.json
../data/result/1203/chunk_summaries\N11_background.json
../data/result/1203/chunk_summaries\N11_topics.json
../data/result/1203/chunk_summaries\N12_background.json
../data/result/1203/chunk_summaries\N12_topics.json
../data/result/1203/chunk_summaries\N13_background.json
../data/result/1203/chunk_summaries\N13_topics.json
../data/result/1203/chunk_summaries\N14_background.json
../data/result/1203/chunk_summaries\N14_topics.json
../data/result/1203/chunk_summaries\N15_background.json
../data/result/1203/chunk_summaries\N15_topics.json
../data/result/1203/chunk_summaries\N16_background.json
../data/result/1203/chunk_summaries\N16_topics.json
../data/result/1203/chunk_summaries\N17_background.json
../data/result/1203/chunk_summaries\N17_topics.json
../data/result/1203/chunk_summaries\N18_background.json
../data/result/1203/chunk_summaries\N18_topics.json
../data/result/1203/chunk_su

In [13]:
# Note: run time very long, consider running on kwon
driven_def = json.load(open("../data/raw/definitions/Drivers_variables.json"))
for interview_file in glob.glob("../data/result/1203/chunk_summaries/*.json"):
    interview_data = json.load(open(interview_file))
    # print(interview_file)
    for chunk in interview_data:
        
        interviewee_messages = "\n".join([message['content'] for message in chunk['conversation'] if message['speaker'] == 0])
        nodes = generate_nodes(interviewee_messages,driven_def, "Drivers")
        # print(interviewee_messages)
        # print(keywords)
        # chunk['keywords'] = keywords
    # save_json(interview_data, interview_file)

ValueError: unexpected '{' in field name

In [5]:
from collections import defaultdict
from pprint import pprint
import re
topic_keywords = defaultdict(lambda: defaultdict(int))
for interview_file in glob.glob("../data/result/chunk_summaries/*.json"):
    interview_data = json.load(open(interview_file))
    for chunk in interview_data:
        keywords_str = chunk['raw_keywords']
        keywords = re.split(", |、", keywords_str)
        keywords = [keyword.replace("\"", "").replace('[', "") for keyword in keywords]
        topic = chunk['topic']
        for keyword in keywords:
            topic_keywords[topic][keyword] += 1
save_json(topic_keywords, "../data/result/topic_keywords.json")

In [ ]:
from collections import defaultdict
import chinese_converter
known_topics = ['整體經濟', '住屋', '公有土地', '貿易', '政府運作', '災害', '醫療', '環境生態', '其他', '交通', '能源']
stopwords = open('stopwords-master/all.txt', 'r', encoding='utf-8').read().split('\n')
stopwords.append("綠島")
# stopwords.append('["綠島"')
stopwords = [chinese_converter.to_traditional(word) for word in stopwords]

In [ ]:
import string
# reverse index: keyword -> topic
keyword_topic_appearances = defaultdict(set)
keyword_frequency = defaultdict(int)
topic_keywords = json.load(open("../data/result/topic_keywords.json"))
for topic, keywords_freq in topic_keywords.items():
    keywords = list(keywords_freq.keys())
    for keyword in keywords:
        keyword_topic_appearances[keyword].add(topic)
        keyword_frequency[keyword] += keywords_freq[keyword]
# sort by number of topics
keyword_topic_appearances = sorted(keyword_topic_appearances.items(), key=lambda x: len(x[1]), reverse=True)
count = 0
keywords = []
for keyword, topics in keyword_topic_appearances:
    keyword = keyword.translate(str.maketrans('', '', string.punctuation))
    if keyword in stopwords: continue
    if keyword in known_topics: continue
    # if len(topics) == 1: continue
    if keyword_frequency[keyword] < 2: continue
    print(keyword, keyword_frequency[keyword], topics)
    count += 1
    keywords.append(keyword)
print(keywords)
save_json(keywords, "../data/result/tmp/keywords.json")

觀光 41 {'環境生態', '整體經濟', '能源', '公有土地', '住屋', '災害', '醫療', '政府運作', '交通'}
民宿 36 {'環境生態', '整體經濟', '能源', '公有土地', '住屋', '災害', '醫療', '政府運作', '交通'}
遊客 44 {'環境生態', '能源', '公有土地', '住屋', '災害', '醫療', '政府運作', '交通'}
土地 22 {'環境生態', '能源', '公有土地', '住屋', '災害', '醫療', '政府運作', '交通'}
臺東 38 {'環境生態', '能源', '公有土地', '住屋', '災害', '醫療', '政府運作', '交通'}
潛水 38 {'環境生態', '整體經濟', '公有土地', '住屋', '災害', '醫療', '政府運作', '交通'}
政府 38 {'環境生態', '能源', '公有土地', '住屋', '災害', '政府運作', '交通'}
工作 21 {'環境生態', '能源', '公有土地', '住屋', '醫療', '政府運作', '交通'}
生態 32 {'環境生態', '能源', '公有土地', '災害', '醫療', '政府運作', '交通'}
規劃 12 {'環境生態', '能源', '公有土地', '住屋', '醫療', '政府運作', '交通'}
環境 24 {'環境生態', '整體經濟', '公有土地', '住屋', '災害', '政府運作', '交通'}
蘭嶼 17 {'環境生態', '能源', '公有土地', '住屋', '災害', '醫療', '交通'}
總體經濟 13 {'環境生態', '整體經濟', '能源', '公有土地', '住屋', '醫療', '交通'}
開發 16 {'環境生態', '公有土地', '住屋', '災害', '政府運作', '交通'}
居民 27 {'環境生態', '公有土地', '住屋', '醫療', '政府運作', '交通'}
縣政府 22 {'環境生態', '整體經濟', '公有土地', '住屋', '政府運作', '交通'}
投資 9 {'環境生態', '整體經濟', '能源', '公有土地', '住屋', '災害'}
賺錢 13 {'環境生態', '公有土地', '住屋', '災

In [ ]:
import json
import glob
import re
candidate_keywords = json.load(open("../data/result/tmp/keywords.json"))
for interview_file in glob.glob("../data/result/chunk_summaries/*.json"):
    interview_data = json.load(open(interview_file))
    for index, chunk in enumerate(interview_data):
        keywords_str = chunk['raw_keywords']
        keywords = re.split(", |、", keywords_str)
        keywords = [keyword.replace("\"", "").replace('[', "") for keyword in keywords]
        keywords = [keyword for keyword in keywords if keyword in candidate_keywords]
        chunk['keywords'] = keywords
        interview_data[index] = chunk
    save_json(interview_data, interview_file)

In [ ]:
keyword_embeddings = json.load(open("../data/result/keyword_coordinates.json"))
if "綠島" in keyword_embeddings:
    print("!!!")
    del keyword_embeddings["綠島"]
save_json(keyword_embeddings, "../data/result/keyword_coordinates.json")

!!!


In [ ]:
keywords = json.load(open("../data/result/tmp/keywords.json"))
keyword_statistics = {}
for keyword, topics in keyword_topic_appearances:
    if keyword not in keywords: continue
    topic_freq = {}
    for topic in topics:
        topic_freq[topic] = topic_keywords[topic][keyword]
    print(keyword, keyword_frequency[keyword], topic_freq)
    keyword_statistics[keyword] = {
        "frequency": keyword_frequency[keyword],
        "frequency_by_topic": topic_freq
    }
save_json(keyword_statistics, "../data/result/keyword_statistics.json")

觀光 41 {'環境生態': 21, '整體經濟': 1, '能源': 1, '公有土地': 1, '住屋': 1, '災害': 1, '醫療': 2, '政府運作': 7, '交通': 6}
民宿 36 {'環境生態': 11, '整體經濟': 1, '能源': 1, '公有土地': 2, '住屋': 7, '災害': 1, '醫療': 2, '政府運作': 3, '交通': 8}
遊客 44 {'環境生態': 23, '能源': 3, '公有土地': 1, '住屋': 3, '災害': 2, '醫療': 3, '政府運作': 2, '交通': 7}
土地 22 {'環境生態': 4, '能源': 1, '公有土地': 2, '住屋': 5, '災害': 1, '醫療': 1, '政府運作': 5, '交通': 3}
臺東 38 {'環境生態': 14, '能源': 2, '公有土地': 3, '住屋': 4, '災害': 3, '醫療': 5, '政府運作': 3, '交通': 4}
潛水 38 {'環境生態': 20, '整體經濟': 1, '公有土地': 1, '住屋': 6, '災害': 2, '醫療': 1, '政府運作': 1, '交通': 6}
政府 38 {'環境生態': 17, '能源': 1, '公有土地': 1, '住屋': 2, '災害': 1, '政府運作': 11, '交通': 5}
工作 21 {'環境生態': 10, '能源': 1, '公有土地': 2, '住屋': 5, '醫療': 1, '政府運作': 1, '交通': 1}
生態 32 {'環境生態': 24, '能源': 1, '公有土地': 1, '災害': 1, '醫療': 1, '政府運作': 2, '交通': 2}
規劃 12 {'環境生態': 5, '能源': 1, '公有土地': 1, '住屋': 1, '醫療': 1, '政府運作': 2, '交通': 1}
環境 24 {'環境生態': 11, '整體經濟': 1, '公有土地': 2, '住屋': 1, '災害': 1, '政府運作': 5, '交通': 3}
蘭嶼 17 {'環境生態': 7, '能源': 1, '公有土地': 2, '住屋': 2, '災害': 1, '醫療': 1, '交通': 3}


In [ ]:
keyword_embeddings = {}
for index, keyword in enumerate(keywords):
    print("{}/{}".format(index, len(keywords)))
    keyword_embedding = get_embedding(keyword)
    keyword_embeddings[keyword] = keyword_embedding
save_json(keyword_embeddings, "../data/result/keyword_embeddings.json")


0/1021
1/1021
2/1021
3/1021
4/1021
5/1021
6/1021
7/1021
8/1021
9/1021
10/1021
11/1021
12/1021
13/1021
14/1021
15/1021
16/1021
17/1021
18/1021
19/1021
20/1021
21/1021
22/1021
23/1021
24/1021
25/1021
26/1021
27/1021
28/1021
29/1021
30/1021
31/1021
32/1021
33/1021
34/1021
35/1021
36/1021
37/1021
38/1021
39/1021
40/1021
41/1021
42/1021
43/1021
44/1021
45/1021
46/1021
47/1021
48/1021
49/1021
50/1021
51/1021
52/1021
53/1021
54/1021
55/1021
56/1021
57/1021
58/1021
59/1021
60/1021
61/1021
62/1021
63/1021
64/1021
65/1021
66/1021
67/1021
68/1021
69/1021
70/1021
71/1021
72/1021
73/1021
74/1021
75/1021
76/1021
77/1021
78/1021
79/1021
80/1021
81/1021
82/1021
83/1021
84/1021
85/1021
86/1021
87/1021
88/1021
89/1021
90/1021
91/1021
92/1021
93/1021
94/1021
95/1021
96/1021
97/1021
98/1021
99/1021
100/1021
101/1021
102/1021
103/1021
104/1021
105/1021
106/1021
107/1021
108/1021
109/1021
110/1021
111/1021
112/1021
113/1021
114/1021
115/1021
116/1021
117/1021
118/1021
119/1021
120/1021
121/1021
122/1021
123

In [ ]:
import numpy as np
import scipy.spatial as spatial
def distance_matrix(embeddings):
    embeddings = np.array(embeddings)
    return spatial.distance.cdist(embeddings, embeddings, metric='cosine')
D = distance_matrix(list(keyword_embeddings.values()))
import math
max = D.max()
D /= max
keyword_0_distances = D[0]
sorted_indices = sorted(range(len(keyword_0_distances)), key=lambda k: keyword_0_distances[k])
print(sorted_indices)

In [ ]:
for i in sorted_indices:
    print(list(keyword_embeddings.keys())[i])

In [ ]:
from minisom import MiniSom    
data = np.array(list(keyword_embeddings.values()))
som = MiniSom(30, 30, 1536, sigma=0.1, learning_rate=0.1) 
som.train(data, 2000) 

In [ ]:
som.winner(data[2])
for keyword, embedding in keyword_embeddings.items():
    print(keyword, som.winner(embedding))

In [ ]:
def remove_stop_words(text):
    stopwords = open('stopwords-master/all.txt', 'r', encoding='utf-8').read().split('\n')
    stopwords = [chinese_converter.to_traditional(word) for word in stopwords]
    for word in stopwords:
        text = text.replace(word, "")
    return text
def explain_keyword(keyword):
    messages = [
        {
            "role": "system",
            "content": """You are a Chinese phrase explainer. You explain the meaning of a Chinese phrase.
                The user will give you a Chinese phrase, please explain the meaning of the phrase in Traditional Chinese.
                Reply with Traditional Chinese.
              """
        },
        {
            "role": "user",
            "content": keyword
        }
    ]
    return request_gpt4(messages)
explanation_1 = explain_keyword("民宿")
explanation_2 = explain_keyword("溫泉")
explanation_3 = explain_keyword("貨櫃")
print("民宿", explanation_1)
print("溫泉", explanation_2)
print("貨櫃", explanation_3)
embedding_1 = get_embedding(explanation_1)
embedding_1_1 = get_embedding("民宿")
embedding_2 = get_embedding(explanation_2)
embedding_2_2 = get_embedding("溫泉")
embedding_3 = get_embedding(explanation_3)
embedding_3_3 = get_embedding("貨櫃")
print(cosine_similarity(embedding_1, embedding_2), cosine_similarity(embedding_1_1, embedding_2_2))
print(cosine_similarity(embedding_1, embedding_3), cosine_similarity(embedding_1_1, embedding_3_3))
print(cosine_similarity(embedding_2, embedding_3), cosine_similarity(embedding_2_2, embedding_3_3))

NameError: name 'request_gpt4' is not defined